<a href="https://colab.research.google.com/github/edcote/kaggle/blob/main/Titantic_Machine_Learning_from_Disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab setup

In [ ]:
!pip install -q kaggle blackcellmagic
import pandas as pd
import numpy as np

In [ ]:
%reload_ext blackcellmagic

# Kaggle setup

In [ ]:
%%black
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c titanic
!unzip -qf titanic.zip

In [ ]:
%%black

train_data = pd.read_csv('/content/train.csv', index_col='PassengerId')
train_data['Age'] =    train_data['Age'].  fillna(train_data['Age'].median())
train_data['Embarked'] = train_data['Embarked'].fillna('S')
train_data['Sex'] =     train_data['Sex'].map({'female': 0, 'male': 1}).astype(int)
train_data['Embarked_S'] = train_data['Embarked'] == 'S'
train_data['Embarked_C'] = train_data['Embarked'] == 'C'
train_data['Embarked_Q'] = train_data['Embarked'] == 'Q'

age_bins = np.linspace(train_data['Age'].min(),
                       train_data['Age'].max(),
                       num=8,
                       endpoint=True)
age_labels = [ f"{age_bins[i-1]:.2f}_{age_bins[i]:.2f}" for i in range(1, len(age_bins)) ]
train_data['Age Group'] = pd.cut(train_data['Age'],
                                 bins = age_bins,
                                 labels = age_labels)

fare_bins = np.linspace(train_data['Fare'].min(),
                       train_data['Fare'].max(),
                       num=8,
                       endpoint=True)
fare_labels = [ f"{fare_bins[i-1]:.2f}_{fare_bins[i]:.2f}" for i in range(1, len(fare_bins)) ]
train_data['Fare Group'] = pd.cut(train_data['Fare'],                                 bins = fare_bins,
                                 labels = fare_labels)

train_data = train_data.drop(['Ticket', 'Name', 'Cabin', 'Embarked', 'Age', 'Fare'], axis=1)

train_data.head()

In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')

#train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
#train_data['Sex'] = train_data['Sex'].map({'female': 0, 'male': 1}).astype(int)
#train_data = train_data.drop(['Ticket', 'Name', 'Cabin'], axis=1)


In [ ]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.head()

In [ ]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

In [ ]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

# Data cleaning

In [ ]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')

#train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
#train_data['Sex'] = train_data['Sex'].map({'female': 0, 'male': 1}).astype(int)
#train_data = train_data.drop(['Ticket', 'Name', 'Cabin'], axis=1)


In [ ]:
train_data.describe()